# 🎭 PART 2: ペルソナライゼーション検証

**目的**: 生成文章が実際にペルソナの特徴を反映しているか検証

**検証方法**:
1. ペルソナプロファイルを読み込む
2. ペルソナの特徴（personality traits, communication style, values）を確認
3. Base/Steering生成文章がペルソナ特徴を反映しているか分析

# Base vs Steering 実際の出力検査

**目的**: 実際の生成結果とJudge判定を確認し、なぜ90.2%が引き分けだったのかを理解する

**確認項目**:
1. Tie（引き分け）のケース - 最も多い（90.2%）
2. Steering勝利のケース - 17/20の勝利（80.3%）
3. Base勝利のケース - 3/20の勝利
4. 効果があったペルソナ vs 効果がなかったペルソナの比較

In [ ]:
import json
import pandas as pd
from pathlib import Path
from IPython.display import display, HTML, Markdown
import textwrap

print("✓ Libraries loaded")

## 1. データ読み込み

In [ ]:
# Load comparison results
with open("../results/base_vs_steering/comparison_results.json") as f:
    data = json.load(f)

results = data["results"]

print(f"Total comparisons: {len(results)}")
print(f"Config: {data['config']}")

# Count by winner
winner_counts = {"tie": 0, "steering": 0, "base": 0}
for r in results:
    winner_counts[r["winner"]] += 1

print(f"\nWinner distribution:")
for winner, count in winner_counts.items():
    print(f"  {winner}: {count} ({100*count/len(results):.1f}%)")

## 2. ヘルパー関数: きれいに表示

In [ ]:
def display_comparison(result, index=None):
    """
    Display a single comparison in a nice format
    """
    # Header
    header = f"## Comparison {index}" if index else "## Comparison"
    display(Markdown(header))
    
    # Basic info
    info_html = f"""
    <div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px; margin-bottom: 10px;">
        <strong>Persona:</strong> {result['persona_id']}<br>
        <strong>Winner:</strong> <span style="color: {'green' if result['winner']=='steering' else 'red' if result['winner']=='base' else 'gray'}; font-weight: bold;">{result['winner'].upper()}</span><br>
        <strong>Confidence:</strong> {result['confidence']}/5.0<br>
        <strong>Judge Winner:</strong> {result['judge_winner']} (Steering is {'A' if result['steering_is_a'] else 'B'})
    </div>
    """
    display(HTML(info_html))
    
    # Prompt
    display(Markdown("### 📝 Prompt"))
    display(Markdown(f"> {result['prompt']}"))
    
    # Base response
    display(Markdown("### 🔵 Base Response (No Steering)"))
    base_html = f"""
    <div style="background-color: #e3f2fd; padding: 15px; border-left: 4px solid #2196F3; margin: 10px 0;">
        {result['response_base']}
    </div>
    """
    display(HTML(base_html))
    
    # Steering response
    display(Markdown("### 🔴 Steering Response (With Optimized Vectors)"))
    steering_html = f"""
    <div style="background-color: #ffebee; padding: 15px; border-left: 4px solid #f44336; margin: 10px 0;">
        {result['response_steering']}
    </div>
    """
    display(HTML(steering_html))
    
    # Judge explanation
    display(Markdown("### 🧑‍⚖️ Judge Explanation"))
    explanation_html = f"""
    <div style="background-color: #fff3e0; padding: 15px; border-left: 4px solid #ff9800; margin: 10px 0;">
        {result['explanation']}
    </div>
    """
    display(HTML(explanation_html))
    
    display(HTML("<hr style='border: 2px solid #ccc; margin: 30px 0;'>"))

print("✓ Display function defined")

## 3. 代表例: Tie（引き分け）ケース

最も多い結果（90.2%）。BaseとSteeringが区別できなかった。

In [ ]:
# Get tie examples
tie_results = [r for r in results if r["winner"] == "tie"]

print(f"Total tie cases: {len(tie_results)}")
print(f"Showing first 5 examples...\n")

for i, result in enumerate(tie_results[:5], 1):
    display_comparison(result, index=f"TIE-{i}")

## 4. 代表例: Steering勝利ケース

Steeringが効果を発揮したケース（decisive comparisonsの80.3%）

In [ ]:
# Get steering win examples
steering_wins = [r for r in results if r["winner"] == "steering"]

print(f"Total steering wins: {len(steering_wins)}")
print(f"Showing all examples...\n")

for i, result in enumerate(steering_wins, 1):
    display_comparison(result, index=f"STEERING-WIN-{i}")

## 5. 代表例: Base勝利ケース

Steeringが逆効果だったケース（decisive comparisonsの19.7%）

In [ ]:
# Get base win examples
base_wins = [r for r in results if r["winner"] == "base"]

print(f"Total base wins: {len(base_wins)}")
print(f"Showing all examples...\n")

for i, result in enumerate(base_wins, 1):
    display_comparison(result, index=f"BASE-WIN-{i}")

## 6. ペルソナ別分析: 効果があったペルソナ

効果があった4つのペルソナの具体例を確認

In [ ]:
# Personas that had effect
effective_personas = ["episode-118328_B", "episode-184019_A", "episode-225888_A", "episode-239427_A"]

for persona_id in effective_personas:
    display(Markdown(f"# ペルソナ: {persona_id}"))
    
    # Get all comparisons for this persona
    persona_results = [r for r in results if r["persona_id"] == persona_id]
    
    # Count outcomes
    outcomes = {"tie": 0, "steering": 0, "base": 0}
    for r in persona_results:
        outcomes[r["winner"]] += 1
    
    total = len(persona_results)
    display(Markdown(f"""
**統計**:
- Total: {total}
- Tie: {outcomes['tie']} ({100*outcomes['tie']/total:.1f}%)
- Steering wins: {outcomes['steering']} ({100*outcomes['steering']/total:.1f}%)
- Base wins: {outcomes['base']} ({100*outcomes['base']/total:.1f}%)
"""))
    
    # Show 2 examples from each category
    display(Markdown("## サンプル例"))
    
    # Steering wins
    steering_examples = [r for r in persona_results if r["winner"] == "steering"][:2]
    for i, result in enumerate(steering_examples, 1):
        display_comparison(result, index=f"{persona_id}-STEERING-{i}")
    
    # Ties
    tie_examples = [r for r in persona_results if r["winner"] == "tie"][:2]
    for i, result in enumerate(tie_examples, 1):
        display_comparison(result, index=f"{persona_id}-TIE-{i}")
    
    display(HTML("<hr style='border: 4px solid #000; margin: 50px 0;'>"))

## 7. 効果がなかったペルソナのサンプル

全てが引き分けだったペルソナの例

In [ ]:
# Personas with all ties
from collections import defaultdict

persona_outcomes = defaultdict(lambda: {"tie": 0, "steering": 0, "base": 0})

for r in results:
    persona_id = r["persona_id"]
    winner = r["winner"]
    persona_outcomes[persona_id][winner] += 1

# Find all-tie personas
all_tie_personas = []
for persona_id, outcomes in persona_outcomes.items():
    total = sum(outcomes.values())
    if outcomes["tie"] == total:
        all_tie_personas.append(persona_id)

print(f"Personas with all ties: {len(all_tie_personas)}")
print(f"Showing examples from first 3 personas...\n")

for persona_id in all_tie_personas[:3]:
    display(Markdown(f"# ペルソナ: {persona_id} (全て引き分け)"))
    
    # Show 3 examples
    persona_results = [r for r in results if r["persona_id"] == persona_id][:3]
    
    for i, result in enumerate(persona_results, 1):
        display_comparison(result, index=f"{persona_id}-{i}")
    
    display(HTML("<hr style='border: 4px solid #000; margin: 50px 0;'>"))

## 8. 文字列差分分析

BaseとSteeringの出力がどれだけ異なるか定量的に確認

In [ ]:
import difflib
import numpy as np

# Calculate similarity ratios
similarities = []

for r in results:
    base = r["response_base"]
    steering = r["response_steering"]
    
    ratio = difflib.SequenceMatcher(None, base.lower(), steering.lower()).ratio()
    
    similarities.append({
        "persona_id": r["persona_id"],
        "winner": r["winner"],
        "similarity": ratio
    })

df_sim = pd.DataFrame(similarities)

print("文字列一致率の統計:")
print(f"平均: {df_sim['similarity'].mean():.3f}")
print(f"標準偏差: {df_sim['similarity'].std():.3f}")
print(f"最小: {df_sim['similarity'].min():.3f}")
print(f"最大: {df_sim['similarity'].max():.3f}")

print("\n結果別の一致率:")
print(df_sim.groupby('winner')['similarity'].describe())

In [ ]:
# Show examples with lowest similarity (most different)
most_different = df_sim.nsmallest(10, 'similarity')

print("最も異なる出力のTop 10:")
print(most_different)

print("\n詳細を表示...\n")

for idx in most_different.index[:5]:
    result = results[idx]
    display_comparison(result, index=f"MOST-DIFFERENT-{idx}")

## 9. まとめと考察

In [ ]:
display(Markdown("""
# 🔍 観察結果

## 主な発見

1. **ほとんどの出力が非常に似ている**
   - 文字列一致率が高い
   - Judgeが違いを検出できない

2. **Steeringの効果が限定的**
   - 84.6%のペルソナで全く差が出ない
   - 重みが小さい（L2 norm < 4）ペルソナで特に効果なし

3. **効果があったペルソナの特徴**
   - L2 norm > 7
   - R2とR4の重みが大きい

## 推奨される改善策

1. **重みを大きくする最適化**
   - 現在の重みは小さすぎる
   - より大きなalphaを試す

2. **より敏感な評価指標**
   - GPT-4oでは微妙な差を検出できない
   - 自動評価メトリクスの導入

3. **Layer選択の再検討**
   - Layer 20が最適か検証
   - 複数レイヤーでのsteering

4. **Adaptive Trait Selection**
   - ペルソナごとに重要なTraitのみを使用
   - 不要なTraitの除去で重みを集中
"""))